In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from db_helper_functions import get_stock_news_from_db
from sklearn.decomposition import NMF, LatentDirichletAllocation, MiniBatchNMF
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bertopic import BERTopic
from finbert_embedding.embedding import FinbertEmbedding
import torch

In [22]:
df = get_stock_news_from_db("AAPL")
df = df[~df.article.isnull()]
t_df = df.iloc[:10]
t_df

,id,ticker,title,url,article,date
0,4235,AAPL,"Microsoft, Alphabet, Meta, Apple, And Amazon L...",https://www.benzinga.com/news/earnings/22/07/2...,(Monday Market Open) Investors appear optimist...,2022-07-25
1,4376,AAPL,Benzinga Before The Bell: More Amazon Workers ...,https://www.benzinga.com/news/22/08/28532545/b...,CNBCBlackRock To Pledge A$1B In Australian Bat...,2022-08-17
2,3,AAPL,"Market Rebounds On Trade Optimism, Tech Bounce...",https://www.benzinga.com/node/12934767,A stronger-than-expected government report on ...,2019-01-04
3,4,AAPL,American Shippers And Carriers React To Signs ...,https://www.benzinga.com/node/12938837,The first week of 2019 saw three of America's ...,2019-01-07
4,5,AAPL,TD Ameritrade IMX Continued Its Dip In Decembe...,https://www.benzinga.com/node/12953011,"Declining for the third month in a row, TD Ame...",2019-01-08
5,6,AAPL,Apple CEO Tim Cook Talks With Jim Cramer—Here ...,https://www.benzinga.com/node/12957963,Apple Inc. AAPL CEO Tim Cook spoke with CNBC's...,2019-01-09
6,7,AAPL,Watching The Clock: Trade Talks End With Optim...,https://www.benzinga.com/node/12958812,"Trade talks are over for this week, and now it...",2019-01-09
7,9,AAPL,TD Ameritrade's Investor Movement Index Highli...,https://www.benzinga.com/node/12960171,Few investors would argue 2018 was a good year...,2019-01-09
8,10,AAPL,"Stung By Poor Fiscal Q2 Print, FedEx Executive...",https://www.benzinga.com/node/12968719,Top FedEx Corp. FDX executives met with securi...,2019-01-10
9,11,AAPL,"Another Day, Another Bank Misses On Earnings: ...",https://www.benzinga.com/node/12986104,Big bank earnings might get overshadowed a bit...,2019-01-15


In [23]:
finbert_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

In [24]:
finbert_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [26]:
only_end_of_articles = [x[-512:] for x in t_df.article]

In [14]:
finbert_mat = finbert_tokenizer(only_end_of_articles, padding=True, return_tensors="pt")

In [17]:
outputs = finbert_model(**finbert_mat)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-1.1004, -0.9528,  2.6073],
        [-0.3714, -1.7940,  2.4050],
        [-0.6886, -1.4726,  2.4825],
        [ 0.4505, -1.5235,  1.0758],
        [-1.1281, -0.4418,  2.0902],
        [-0.0192, -1.8209,  1.9907],
        [-0.9612, -1.1091,  2.4879],
        [-0.3719, -1.4385,  2.0067],
        [-1.5400,  1.9244,  0.4078],
        [-1.0008, -1.1154,  2.5392]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [28]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions.tolist()

[[0.023300036787986755, 0.027004476636648178, 0.9496954679489136],
 [0.05779491364955902, 0.013933811336755753, 0.9282712340354919],
 [0.03954237699508667, 0.0180546585470438, 0.9424030184745789],
 [0.33248171210289, 0.04618183523416519, 0.6213364005088806],
 [0.035750437527894974, 0.07101218402385712, 0.8932373523712158],
 [0.11590597033500671, 0.019126292318105698, 0.8649677634239197],
 [0.029998810961842537, 0.025873176753520966, 0.9441280364990234],
 [0.08240769058465958, 0.02836253121495247, 0.8892297744750977],
 [0.025017259642481804, 0.7995222806930542, 0.17546039819717407],
 [0.02750471979379654, 0.024526527151465416, 0.9479687809944153]]

In [29]:
t_df[["positive", "negative", "neutral"]] = predictions.tolist()

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.